## Обучение многослойного перцептрона для выполнения среды LunarLander на основе фреймворка Keras

**Импортирование библиотек**

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "torch"

import gymnasium as gym
from gymnasium.wrappers import RecordVideo
import numpy as np
import keras
from keras import ops, layers
from torch import optim, nn
import torch
import matplotlib.pyplot as plt
import onnxruntime
print("Все библиотеки загружены!")

**Конфигурация параметров**

In [ ]:
seed = 42       # Псевдо-случайный генератор
gamma = 0.99    # Коэффициент дисконтирования прошлых вознаграждений
max_steps_per_episode = 1000  # Максимальное кол-во циклов для одного эпизода
env = gym.make("LunarLander-v3")  # Создание среды
env.reset(seed=seed)    # Инициализация первого кадра
eps = np.finfo(np.float32).eps.item()  # Очень маленькое число чтобы избежать деления на ноль
print("Среда создана и параметры заданы!")

**Реализация сети Actor Critic (https://arxiv.org/pdf/1602.01783v2)**

Эта сеть выполняет две функции:

1. Действующий агент (Actor): Принимает на вход состояние окружающей среды и возвращает значение вероятности
для каждого действия в пространстве действий.
2. Агент-критик (Critic): Принимает на вход состояние окружающей среды и возвращает
оценку общего вознаграждения в будущем.

В нашей реализации они совместно используют начальный слой.

In [ ]:
num_inputs = 8      # Кол-во входов
num_actions = 4     # Кол-во действий
num_hidden = 32    # Кол-во нейронов в скрытом слое

# Конфигурация модели
inputs = layers.Input(shape=(num_inputs,))
common = layers.Dense(num_hidden, activation="relu")(inputs)
action = layers.Dense(num_actions, activation="softmax")(common)
critic = layers.Dense(1)(common)

actor_model = keras.Model(inputs=inputs, outputs=action)
critic_model = keras.Model(inputs=inputs, outputs=critic)
actor_model.summary()
critic_model.summary()

**Обучение**

In [ ]:
# Создаем оптимизатор и функцию потерь для модели
actor_optimizer = optim.Adam(actor_model.parameters(), lr=5e-3)
critic_optimizer = optim.Adam(critic_model.parameters(), lr=1e-3)
huber_loss = keras.losses.Huber()
# Сохраняем историю для последующего анализа
action_probs_history = []
critic_value_history = []
critic_losses_history = []
actor_losses_history = []
rewards_history = []
running_reward_log = []
running_reward = 0
episode_count = 0

while True:  # Запускаем процесс до остановки одним из условий
    state = env.reset()[0]
    episode_reward = 0
    for timestep in range(1, max_steps_per_episode):

        state = ops.convert_to_tensor(state)
        state = ops.expand_dims(state, 0)

        # Прогнозирование вероятности действий и оценки будущего
        # вознаграждения на основе состояния окружающей среды
        action_probs = actor_model(state)
        critic_value = critic_model(state)
        critic_value_history.append(critic_value[0, 0])

        # Выборка действий из распределения вероятностей действий
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        action_probs_history.append(ops.log(action_probs[0, action]))

        # Применяем выбранное действие в нашей среде
        state, reward, terminated, truncated, _ = env.step(action)

        rewards_history.append(reward)
        episode_reward += reward
        # Если ракета упала, улетела за пределы видимости или вышло время
        if terminated or truncated:
            break

    # Обновляем вознаграждение за выполнение, чтобы проверить условие для решения
    running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward
    running_reward_log.append(running_reward)

    # Вычисляем ожидаемое значение от вознаграждений
    # - На каждом временном шаге каково было общее вознаграждение, полученное после этого временного шага
    # - Вознаграждения в прошлом дисконтируются путем умножения их на гамму
    # - Это метки для нашего критика.
    returns = []
    discounted_sum = 0
    for r in rewards_history[::-1]:
        discounted_sum = r + gamma * discounted_sum
        returns.insert(0, discounted_sum)

    # Нормализуем
    returns = np.array(returns)
    returns = (returns - np.mean(returns)) / (np.std(returns) + eps)
    returns = returns.tolist()

    # Вычисляем значения потерь для обновления нашей сети
    history = zip(action_probs_history, critic_value_history, returns)
    actor_losses = []
    critic_losses = []
    for log_prob, value, ret in history:
        # В этот момент истории критик предположил, что в будущем мы получим
        # общую награду = `value`. Мы совершили действие с вероятностью
        # от `log_prob` и в итоге получили общую награду = `ret`.
        # Актёр должен быть обновлен таким образом, чтобы он предсказывал действие, которое приведет к
        # высокой награде (по сравнению с оценкой критика) с высокой вероятностью.
        diff = ret - value.detach()
        actor_loss = -log_prob * diff
        actor_losses.append(actor_loss)  # actor loss

        # Критик должен быть обновлен таким образом, чтобы он предсказывал более точную оценку
        # будущих вознаграждений.
        critic_loss = huber_loss(ops.expand_dims(value, 0), ops.expand_dims(ret, 0))
        critic_losses.append(critic_loss)

    # Обратное распространение
    critic_optimizer.zero_grad()
    sum(critic_losses).backward()
    critic_optimizer.step()
    
    actor_optimizer.zero_grad()
    sum(actor_losses).backward()
    actor_optimizer.step()

    critic_losses_history.append(sum(critic_losses).detach().numpy())
    actor_losses_history.append(sum(actor_losses).detach().numpy())

    # Эпизод закончился поэтому очищаем историю
    action_probs_history.clear()
    critic_value_history.clear()
    rewards_history.clear()

    # Логируем прогресс
    episode_count += 1
    if episode_count % 25 == 0:
        template = "Промежуточная награда: {:.2f} на эпизоде {}"
        print(template.format(running_reward, episode_count))

    # Условие, при котором задача считается решенной
    if running_reward > 250:
        print("Решили на эпизоде {}!".format(episode_count))
        break

**Тестирование**

In [ ]:
# Создаем график по накопленным данным, усредняя кривую по 20 значениям
rolling = 20
mov_avg_ret = np.convolve(running_reward_log, np.ones(rolling)/rolling, mode='valid')
mv_actor = np.convolve(actor_losses_history, np.ones(rolling)/rolling, mode='valid')
mv_critic = np.convolve(critic_losses_history, np.ones(rolling)/rolling, mode='valid')
# Рисуем графики прогресса
fig, axs = plt.subplots(2,2, figsize=(12,8))
axs[0,0].plot(mov_avg_ret)
axs[0,0].set_title('Награда за эпизод (ср. скользящая)')
axs[1,0].plot(mv_critic)
axs[1,0].set_title('Critic Loss (ср. скользящая)')
axs[1,1].plot(mv_actor)
axs[1,1].set_title('Actor Loss (ср. скользящая)')
plt.tight_layout()
plt.savefig("graph.png")

# Проверяем работу нашего агента и записываем видео
print("Начинаем тестирование!")
env = gym.make('LunarLander-v3', render_mode='rgb_array', max_episode_steps=500)
env = RecordVideo(env, video_folder="videos", name_prefix="lander_test")
state, _ = env.reset()
done = False
# Экспортируем в формате onnx
actor_model.export("model.onnx", format="onnx")
ort_session = onnxruntime.InferenceSession("model.onnx")
while not done:
    input_name = ort_session.get_inputs()[0].name
    output_name = ort_session.get_outputs()[0].name
    action = ort_session.run([output_name], {input_name: state[None, :]})[0]
    state, reward, terminated, truncated, _ = env.step(np.argmax(action))
    done = terminated or truncated
env.close()

Если все блоки выполнились успешно, поздравляем! Вы получили обученную модель (model.onnx), график обучения (graph.png) и видео с работой вашего агента (videos/lander_test-episode-0.mp4). Видео можно посмотреть локально на компьютере, скачав его через контекстное меню. Теперь, загрузите модель *model.onnx* на сайт http://deepcode.ci.nsu.ru для регистрации в таблице участников. Однако чтобы добиться лучшей точности, вы можете попробовать советы из списка ниже или самостоятельно изменить код обучения.

### **Простые улучшения:**
- Увеличьте время обучения модели.
- Измените расчёт значения для функции потерь, например добавив штраф за долгое выполнение.
- Настройте скорость обучения оптимизатора.
- "Усильте" сеть большим количеством нейронов.
- Попробуйте создать разные модели для актёра и критика.

### **Сложные улучшения:**
- Добавьте градиентный клиппинг чтобы избежать "взрывов" градиентов.
- Увеличьте энтропию на первых порах обучения.
- Векторизуйте среду для обучения параллельных агентов.
- Используйте другие имплементации политик.